### Stappenplan
Data splitten op verschillende tijdsranges en dan de volgende features uitrekenen:
- Number of events in each sub process
- Number of events with each document type
- Average, minimum and maximum amount of time spent in a sub process 
- Average minimum and maximum amount of time spent in a document type
- For each sub process: amount of time spent in the sub process
- For each document type: the time spent in such document type
- The number of days until a first decision is made
- Amount of (un)successful events until a point
- Total time passed
- Month started

In [1]:
import pandas as pd
import numpy as np
import utils
import pyarrow
from datetime import datetime, timedelta

READ_PARQUET = True
pd.set_option('display.max_columns', None)

In [2]:
import importlib
utils = importlib.reload(utils)  # reloads the utils module without restarting kernel

In [3]:
if READ_PARQUET:
    print("Reading parquet.")
    log = utils.load_parquet()
else:
    print("Reading csv and savings as parquet.")
    df = utils.load_csv()
    utils.save_parquet(df)

Reading parquet.


In [4]:
from copy import deepcopy
# drop uncomplete cases (2017 mostly) and thus do not have a complete label

#cases_df = deepcopy(df.loc[df['Complete Timestamp'].dt.year < 2017])
df = deepcopy(log)

In [5]:
cases = pd.DataFrame(df['Case ID'].unique())
cases = cases.rename(columns = {0: 'Case ID'})

In [6]:
df.head()

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,(case) applicant,(case) application,(case) area,(case) basic payment,(case) cross_compliance,(case) department,(case) greening,(case) number_parcels,(case) payment_actual0,(case) payment_actual1,(case) payment_actual2,(case) payment_actual3,(case) penalty_ABP,(case) penalty_AGP,(case) penalty_AJLP,(case) penalty_AUVP,(case) penalty_AVBP,(case) penalty_AVGP,(case) penalty_AVJLP,(case) penalty_AVUVP,(case) penalty_B16,(case) penalty_B2,(case) penalty_B3,(case) penalty_B4,(case) penalty_B5,(case) penalty_B5F,(case) penalty_B6,(case) penalty_BGK,(case) penalty_BGKV,(case) penalty_BGP,(case) penalty_C16,(case) penalty_C4,(case) penalty_C9,(case) penalty_CC,(case) penalty_GP1,(case) penalty_JLP1,(case) penalty_JLP2,(case) penalty_JLP3,(case) penalty_JLP5,(case) penalty_JLP6,(case) penalty_JLP7,(case) penalty_V5,(case) penalty_amount0,(case) penalty_amount1,(case) penalty_amount2,(case) penalty_amount3,(case) program-id,(case) redistribution,(case) rejected,(case) risk_factor,(case) selected_manually,(case) selected_random,(case) selected_risk,(case) small farmer,(case) year,(case) young farmer,activity,concept:name,docid,doctype,eventid,lifecycle:transition,note,subprocess,success
0,8b99873a6136cfa6,Payment application-Application-mail income,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail income,mail income,-18008611495569447,Payment application,NaN,complete,none,Application,True
1,8b99873a6136cfa6,Payment application-Application-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008611495569447,Payment application,NaN,complete,none,Application,True
2,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True
3,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True
4,8b99873a6136cfa6,Parcel document-Main-initialize,fb5fa8,2015/06/10 11:16:28.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,Fa

### Adding Main Process

In [7]:
def calc_events_per_main_process(data, level = 0):
    data['Main_Process'] = data['Activity'].apply(lambda x: x.split('-')[level])
    return data

In [8]:
calc_events_per_main_process(df)


,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,(case) applicant,(case) application,(case) area,(case) basic payment,(case) cross_compliance,(case) department,(case) greening,(case) number_parcels,(case) payment_actual0,(case) payment_actual1,(case) payment_actual2,(case) payment_actual3,(case) penalty_ABP,(case) penalty_AGP,(case) penalty_AJLP,(case) penalty_AUVP,(case) penalty_AVBP,(case) penalty_AVGP,(case) penalty_AVJLP,(case) penalty_AVUVP,(case) penalty_B16,(case) penalty_B2,(case) penalty_B3,(case) penalty_B4,(case) penalty_B5,(case) penalty_B5F,(case) penalty_B6,(case) penalty_BGK,(case) penalty_BGKV,(case) penalty_BGP,(case) penalty_C16,(case) penalty_C4,(case) penalty_C9,(case) penalty_CC,(case) penalty_GP1,(case) penalty_JLP1,(case) penalty_JLP2,(case) penalty_JLP3,(case) penalty_JLP5,(case) penalty_JLP6,(case) penalty_JLP7,(case) penalty_V5,(case) penalty_amount0,(case) penalty_amount1,(case) penalty_amount2,(case) penalty_amount3,(case) program-id,(case) redistribution,(case) rejected,(case) risk_factor,(case) selected_manually,(case) selected_random,(case) selected_risk,(case) small farmer,(case) year,(case) young farmer,activity,concept:name,docid,doctype,eventid,lifecycle:transition,note,subprocess,success,Main_Process
0,8b99873a6136cfa6,Payment application-Application-mail income,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail income,mail income,-18008611495569447,Payment application,NaN,complete,none,Application,True,Payment application
1,8b99873a6136cfa6,Payment application-Application-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008611495569447,Payment application,NaN,complete,none,Application,True,Payment application
2,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True,Entitlement application
3,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True,Entitlement application
4,8b99873a6136cfa6,Parcel document-Main-initialize,fb5fa8,2015/06/10 11:16:28.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

### Adding start date to df

In [9]:
start_date = df.groupby(by = ['Case ID'])['Complete Timestamp'].agg(['min']).rename(columns={'min':'StartDate'})

In [10]:
start_date = start_date.reset_index()
start_date

,Case ID,StartDate
0,0002505cb62792e4,2015/04/28 00:00:00.000
1,0002a55a6130cec8,2015/04/17 00:00:00.000
2,0004ff62053a60ce,2015/05/11 00:00:00.000
3,000612b48d30de74,2015/05/06 00:00:00.000
4,0006cc909ce508b0,2016/04/28 00:00:00.000
...,...,...
43804,fffa80507910b904,2017/05/10 00:00:00.000
43805,fffa93db29ee0841,2016/05/12 00:00:00.000
43806,fffab10d958a3e4d,2016/05/11 00:00:00.000
43807,fffabee101da7ecb,2017/05/11 00:00:00.000


In [11]:
df = pd.merge(df, start_date, on = 'Case ID')
df['Complete Timestamp'] = pd.to_datetime(df['Complete Timestamp']).dt.date
df['StartDate'] = pd.to_datetime(df['StartDate']).dt.date

### Calculate events per sub-process

In [12]:
df.head()

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,(case) applicant,(case) application,(case) area,(case) basic payment,(case) cross_compliance,(case) department,(case) greening,(case) number_parcels,(case) payment_actual0,(case) payment_actual1,(case) payment_actual2,(case) payment_actual3,(case) penalty_ABP,(case) penalty_AGP,(case) penalty_AJLP,(case) penalty_AUVP,(case) penalty_AVBP,(case) penalty_AVGP,(case) penalty_AVJLP,(case) penalty_AVUVP,(case) penalty_B16,(case) penalty_B2,(case) penalty_B3,(case) penalty_B4,(case) penalty_B5,(case) penalty_B5F,(case) penalty_B6,(case) penalty_BGK,(case) penalty_BGKV,(case) penalty_BGP,(case) penalty_C16,(case) penalty_C4,(case) penalty_C9,(case) penalty_CC,(case) penalty_GP1,(case) penalty_JLP1,(case) penalty_JLP2,(case) penalty_JLP3,(case) penalty_JLP5,(case) penalty_JLP6,(case) penalty_JLP7,(case) penalty_V5,(case) penalty_amount0,(case) penalty_amount1,(case) penalty_amount2,(case) penalty_amount3,(case) program-id,(case) redistribution,(case) rejected,(case) risk_factor,(case) selected_manually,(case) selected_random,(case) selected_risk,(case) small farmer,(case) year,(case) young farmer,activity,concept:name,docid,doctype,eventid,lifecycle:transition,note,subprocess,success,Main_Process,StartDate
0,8b99873a6136cfa6,Payment application-Application-mail income,0;n/a,2015-05-08,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail income,mail income,-18008611495569447,Payment application,NaN,complete,none,Application,True,Payment application,2015-05-08
1,8b99873a6136cfa6,Payment application-Application-mail valid,0;n/a,2015-05-08,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008611495569447,Payment application,NaN,complete,none,Application,True,Payment application,2015-05-08
2,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015-05-08,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True,Entitlement application,2015-05-08
3,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015-05-08,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True,Entitlement application,2015-05-08
4,8b99873a6136cfa6,Parcel document-Main-initialize,fb5fa8,2015-06-10,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [13]:
def events_per_sub_process(data, days):
    subprocesses= list(df['subprocess'].unique())
    for index, rows in data:
        case = rows['Case ID']
        subset = df[df['Case ID'] == case]
        subset = subset[subset['Complete Timestamp'] <= (subset['StartDate'] + timedelta(days))]
        for subprocess in subprocesses:
            count = 0 
    #filter such that all activities occur within specified amount of days
    #count amount of occurences for this sub_process
    #put it in a nicely and automatically generated column

In [14]:
subprocesses= list(df['subprocess'].unique())
case = '8b99873a6136cfa6'
subset = df[df['Case ID'] == case]
subset = subset[subset['Complete Timestamp'] <= (subset['StartDate'] + timedelta(90))]
subset['subprocess'].isin(subprocesses).value_counts()

#filtered[filtered.isin(flags)].value_counts().reindex(flags, fill_value=0)
        

True    21
Name: subprocess, dtype: int64

In [15]:
# test = df.loc[df['Case ID'] == 'fffa80507910b904']
# # df.loc[df['column_name'] == some_value]
# test
for index, rows in cases.iterrows():
    print(rows['Case ID'])

8b99873a6136cfa6
d038e6a5fcaed7a4
d0c4a4241daa1d89
beb37ac32c4dfd60
accaf25e4ea7629a
314f58d466e29977
f2e0550f00b5373b
cf28ee3fa1c5d463
1fa3097a021d75ec
c1bad04d1f51af04
894c9d3d58731907
dc8462aad587ab02
cd349f82d78c6ca3
94dcd927bc783fc8
6ef09a7c1107e1b8
6a64ce3f5768249b
501f541a53f6f861
862b21c27faf4354
43385a6115c7a7b8
05e971c42e077a60
dddc216b51e073b4
139ce87f8bb4d9b5
4253b92b93767834
7078c97ba9048a74
ad9d51caaf366a93
8f5be5c99810f84d
9f9fef82e273d0de
74bc83b5aa287fc1
ecc4451ad4d44b10
8363546dca3c20fb
ff073f9c0c122408
573f0e7cd2c59dfd
e5c884456a23c22d
82b330c1cc6107ea
1e8e760e40ca3a59
cbfef65244f0298b
d03bfd1fdbea2761
ce4d4a1b57eeaee3
6186687c81469fd0
fd76f04e8618f92b
a83b71df0291eab7
8de2ff3f8d80bc60
774e8a6847c90293
ff8b5367defe95b3
249e5790e426a6fb
e2816c7186835cd1
c813a964487d8009
644adc0c58c66b30
10e4a357f3ff1711
fc310af90cd07de3
e7aac0cfd16ddc26
0edc87a764b34e67
1baf10de247c5718
e195f03f6d352f8d
9b205e21595a56e7
6bb288754861fe24
184ae8278bed7c92
0535aab96b40fed8
400208769c0cf2

86f3d56769971f07
4ca58dcab3f51bd6
23b7e87714068ade
28633d8289ead57e
ca3f39b3ca59e40f
073e1c7b796df22b
4ff83fbe778f41bf
973f59ed049b3e73
721fb4c5bbfe2f72
d377642c8b46b045
da8d3ddb4a89a4f3
2f16dca36eb6f2c9
25622e285acd76b0
da999bb6dcf81d69
da509e1b06183036
055ac1b34af1082b
85c163250c0035d8
11bee3bff3863b12
0a2385b7c5c22a8b
89cdec94e2238d0e
c754831393071dd5
d0c9c8f95f7be98e
f64862709e9db60d
1db9daf2f338ec75
01d56995450a4b58
f64646475916874d
db8fe2697aed4cc9
eaa009fb21533b5c
647782c69be1292d
2da032900e933a00
8ac3c08fa118e0e9
2c38d5ead4a73ed0
315720c37a0ab3c0
b49ea26f44159834
e94ac86675cf8429
03d5531fd411c664
69bba65264a50793
9da2906c5b8bb4a0
9c1ffa0dc6d4db6e
c3b4df1b28ba4be8
762f713b0f0bd12b
530fdbef1623ec0f
f069e8528bc9d751
f42d76d23c4fc9bd
535992c65f8231bf
957bcfc183673887
d94b029be0ee9d03
f9723bfc76520316
5048a54048c6e0b3
342016e3ed0d26c9
7614e12d6ff79792
431137ec542815ca
c875581bbe2a4b1e
d4ec2be3eea4ca36
871bbde5c1aaa642
837fe686aa668325
2ca7eee99bd76ad0
fecdeaba331d65a2
ca7784cf844d4f

eb395801e383bc4c
016175873ee3f34c
437a7a81689e949d
0dff2e2407841dff
a85f445e014a9d75
156e79f3d1ce58e1
11c81fa599bf15ed
c969777e1d1c9a1e
bbe987113ac9d015
f2e277469b4c76ff
d91b707dc1e581ef
d0539000a892c9fb
6e5e6c0ed4106b68
ccb7658e1796c2c2
343634fb5f5e8814
8a3c2fc437b84dab
74d5a17bd2657eb4
3247fee2582bfb88
4d113278b5eb85ee
c43cacbed1b8454b
e15013bb17f7dfee
8a87288238cc471d
6147cf506a3cfc7b
2b83270eb2703cfc
d6abe80389a8e14c
71fe97d3b66031a8
21dbc9eb921be8d9
1f4c181f1e46014c
33939c3b7fb21545
d7696886e371a14c
5530caada532509e
3dd61a4848edd93e
aeede5eb07a22f10
a624e2e7282e01fe
66ff6a47ee2e0c8a
949bfbce0a156a9e
3de07e605de0ff2e
1bf0c93825caf1e8
982e818b517a827f
13659846a0d5b95a
0158e76f40c6e84d
6e436949a8d22e4c
b0cdf64248853693
45a22da502cd527e
7a6fb70c5842e486
8d2751cd6e45ac07
42196720738c056a
9d5997f6fd6edb87
d126e787b08fcf70
6006e906cf248287
8012fbc43f1ba9bd
66c90e7df3224187
25affeeff0a99367
5c51b3137371edbe
4a9acc5059b9f46a
c7be2a7ddfea1b08
7c9dd25a1a507e24
b572774bb407ce53
90623276fa65b8

a39d159cb3ad5d37
bc5a560d34bbe542
5ab44a670b5089e7
e73bf375f8ac0e0f
61d79ad8cc68ffc4
5e46839cdbb39ba5
19d2e3e10841627f
ab3fe043e00b8d5b
830cf34d8b89f5a4
75a8df7fc4e3bffd
922011eae4f6ea69
b5829b6d858a0a0a
e82cdec18cecbb75
83ba7b76b4cb5144
adeeb83f138fb3ba
bab549d0c4385bfa
a719ac233be18cee
addd7d3cfffd8a44
0ef5f6a96025bc69
bcb6e2165a85c4b4
4e1352c1f75a49aa
8959e5a9f44df8a2
acfdd2061135eeea
a1cfe706562f6bfb
6740d468e4599ef8
3c6a2554a40fa38c
b4b50dcde546b5bd
729b75468fb9e854
799d2fba5f730c49
2cc9c471afe465f7
0c2e45bd28e84d86
c9f2bdd82978f750
35b22a6b2d5e406a
2aed8dc7a8598f1b
c6a36b919d6361f1
871384242cac0c97
aec5919e2404a10d
6bea3c1fd7a4f9a6
9d3a4eab152358ff
b0f7ddb757cac0d2
09e4819865223d83
7438ca9230481d71
116c51dea984a11d
16519f7c500160f9
6365fee377ace674
47b6860a2eeec007
2c4969eedcfb2289
eb54ab00030591ae
4350ce372ee55cc0
ac3f5db4f7a98024
863045042b6fc540
203f8dc03b9fc314
3ff69b6c75fa2b32
4a6ed704ab4ffab3
805eb0cd02237474
8582ff91f438c961
9277df1aba29c940
be37f2fe063c061f
5ceab127a2ec35

b2586eabe783fb52
b601e1304e0d82bd
ff044d5313851d34
8a3137f2ca362acd
a6c7cc59d52ee751
b83c51c6b624e76e
44fe83662461325a
496bdccceb9f914a
2f7babb8ea67ce08
f4b364ec4b6f7331
a81590d6a290a7fc
44a11b556417a326
ad89ac38777976cd
ec3afdbd0bfc559a
3561ceee6e231fbb
9679a29995249329
87b22eaaa36882b7
02c4ff20d1195196
eace36df0d4bd9b3
c015282d329b404d
0e5871fc8abee331
789c180807473db9
2202b247728435e3
1518438d49f9f235
a3cfa553681b17f4
990d5a4ae42c13f4
f4904ea864abfba6
ac8a7f6bb2de8d56
66ec5cf5b6b7f538
dad49f6f2140736b
f75b377d60cc0d77
f9551c871d625d87
0ba9a6c3039282a3
7b11700bfa3282c9
92f26d79dfb134c5
8a33b93da76c376a
8d4c013db2c71d4a
20ce3116d8e94318
49124905f9222ef0
4311957b49d777a8
93ffe1393484e1bb
2f19f87b5912c545
546d49bd20b34e37
5f7dde125addedb8
a396ee1f734f3f2a
781d231fee58d790
608559c546cc0c34
c508e1f0997cb3ca
1aa1d6dd9d9e7606
6c71092a5ff8b777
6a23723ae81786fb
00188e4207f9472d
b42b7e7b04e9742f
b74ffd453e53b179
6db5f78c3702800b
55847e01680f1c19
7dc8516a676566be
3c9c25e5a3571478
c0cd76bb59ca60

...

88755bb4bb4ee6cd
4e8ee07ca448d37a
80d8597ef206e826
99ff699bdc3f48a4
c0dc69f7f1ebe104
6eab5f0943abad9b
68c4c782cedd0b23
b21e90f54c8eda74
64cbffa6962980dc
41e1ebb45fa819c7
ebeea9510a382471
14700a22dd65a9c7
bf10b53b46318dc0
fdd924f5c137d838
1391ea30f254f859
e05654ef24edeacb
17dea450f3ab964a
f830db8b6162c6ef
d2ccc7518b802087
241df99b50413f86
ab953c71ebffea99
58337792ba2ceb17
47c6d4569d772559
c85ed23ffb9961d5
d99484b5a78b9cc4
3ab2ab644a28b1d3
2ecc6aae0d6c33a9
78d993cad06b0a66
fcdec1948cf84c54
af5d0d77e5d23fb7
5049ce43dded10a3
0f79383094d0bacc
e9194784d716748d
7370096ba811f982
259865376de0c40e
48f6d755f7c1f744
169d43702e351086
c9e121d2da5f84fd
94b7894a031098cd
79e93401e7b76a33
17afe3700cab27b7
9b340eb86c307997
6b5cb39fb4ba1412
2003de33f711bb9b
5a7d89e682729513
94014acbc853e544
efb8a82563151175
d350309eebea01c7
a75bf0c157925dc5
5eeb526c34cee87c
e0c56caf3bee4422
bc095e04fdf63196
d5bea2aa97436302
e1d3affe118ec502
3b7df55863b7a646
45acb3243b5a78b5
ca5b4be437c96627
dd4747129efd9989
517ab3d151f679

4396b09e00d4097c
052074129186d048
fe77d1074683d23f
765cfe95de7d0797
5c6f2d961eb42b29
45f51c63e8e541c1
65117bffc0928750
aaa2fd5e56045e24
00e9bb4cecbf951a
299785baf2429e0a
9cadd0e65796420f
90a74d8d0e7dcffe
d23162946ef1336d
ae71b24cffa168e6
bbd13d7772d923bc
2f4648d5303d724a
e5ed222576a6efe3
57e91e98ff4c1cac
8de25c951ba23fe2
0e60b4bbe3c9d260
a568723f35c675ab
f2863713426cfd40
7b8b5875a3d5520a
20f7a95e1663770b
b0ff8978ffd4bde4
2623d757e7c105cf
764a282cf58ed068
1264c20aa5ea1ec9
f877ec5ca136ac05
02f34d9f3ab80eae
1e78761c9af3eec6
58633b661500e11f
18ea80b1f55f02c4
d67ca657326d8dd7
762d8dc2a5e9b6a0
a8370db621e2c412
9383a2eb4ed3a2f5
1d84b7c51ac06a51
40f69eb897853ea9
b892adc38ff97020
2d8eb1f209c27d49
d4a1f24f3f46cb52
83f6b8ab753efe68
238a1b989e2152ee
e19ba2fecf1df6bd
7b0b77baeae19735
be79f193198462bd
8f550f0ca3afff58
7b804b0c6b1dbc7a
3090f30f123940d2
92ef37290ac16d59
d6995e45d6773c1f
6ee31eaeecc4b82a
b5b7d08c6c580693
8b28064d65238144
784c78d3fd36be6c
a5662ae8ba686b18
348888c514cb19b7
136718b920878a

743f9703e6ad0579
3d76b2affc966adf
9c634de7842f9924
7d0f7d00480b0ccd
75a34af0f3a53d21
8856061c1e44886b
d6e7910f55f03cba
4eb189180c77f79b
e9994789bc03e555
b0ab3c7fd76f47ba
7209db73dc92ee1e
d4ec8f399a55d75d
dbf1ab4b59d4c2c0
a3b87af03b63bc7e
fb0bb56196f3170a
eb14797be319dc25
0ad31fc64af1212f
d6db71b5a555fabe
293704b5bc9e6952
51f0438fda37feb6
0670a5ddd0a3309d
5294206adfd32736
dd0abb173399a80b
10e10797c9706c39
ed7212abfef828a5
109d7b2505f25d8e
fa40df5e1533dbdb
e4e0b871ce9f6a27
1e1e0ed42b898bee
19002f3ae0b0bda6
4526a3f3c0acf677
08755d4f51a4dad9
d1c42c448197c240
ede9c969d8bfb53d
ad47dbe50719e76e
b4d1a7a854677514
2f090e26bfb6eb6c
94e9d3f3b7c4525a
5d723bb3fc6a9ca4
c9038915a4748d28
b5b33e6b32810160
ecf2dd8e6120ad46
8a2e4ec265a4de9e
a41d92a381b4a955
621f8d5e8123bc00
3d9c011e1b05d082
5f8a3b978d53415b
4ac471a4ee090c9c
c6fdf93bfc7fc848
e28228dfea656f54
b77a142a0ee78e3c
94cb6cd13a9a7b5a
e919f60cea87478e
15f695497a0c89d9
f6f48ee827b2cbbc
70f5f8768193a6f0
957439b19607bb84
3e6a8a1e6e50b5cb
a82a9620f92599

3ba0ed58fd27f9f0
87b28540836f17f3
0db5479f3eabe503
d49ad671efaccac9
103a389758800a93
65b61e859c453aed
1f95ad0643c9acf7
620923d01001e4b1
1ec837509e94a1d5
289fb97a7b3d79c2
d7464fa4930dd33c
0e2b77be826ab2a7
84c111f9f9e7d505
174aeecc051e5d56
233e35523d2c4c54
4388c7351b3641c7
3d0213f57d39f6b7
985e6306745eae6f
bf439113eaeeb43c
7e9d0d4e4782f635
bad1c2573b10afcb
fc92b076a21dc23b
bcfedab9d9b1310b
db6e5f65432f8b22
c6dcee2cfc084e2a
0bf4e396a997a576
9f2f8a2f96129352
85ca30d131d464a0
e18a3b54bda29db4
90ea022d303aea3e
c345af4663d85d2d
a39bf908cb09ed32
bf18cf17ca2de524
9d112d038ec07aa3
5056f809409227c9
cb7d60d5cc6df3aa
f463daec86765a93
ac35282154b5fd93
ec8390be54b51ccd
ccd076094aaa6a5c
e0d4c8bb3f271174
b489ca25e1d7b069
80f8d6548bb3c744
b0ecf410033b6f0a
4897bdbd1056c9a1
2a565dc24fcb4d24
d7d282dcb61d4ee1
260c1d86a41ab77e
d7b72048ee6a30f7
c8d0363c0423a9a5
bc2dfd67d5a07b12
72d25a2c2e846c65
aa778428cbfeb01c
6f81ed6738bcd54f
937b9e335ac4b5a4
c134b6eecf649fcf
e46da95cf03122d5
a12b8106542d7b3a
bb50279e8050c4

0c162e1c07b08675
fd35af291abf3041
c710c44b1d945392
e5f3a76161abe681
f31947304efa138c
5e2b24b5487e0006
9103b620c72afab7
076a3b8b4f0e504e
e31d23c59399b1cb
f9eee58978c91a11
056e76f9b5712d34
179293aae1f7be8b
7d35885e8cec275c
9a4da90e0db21a50
3e842c29e09c310b
56d0db0a6e7afb8d
00534944a4a2a8a3
401e3e9ce4ca39ec
2c17735834c3bd3c
5ba7057563a0d14f
187fab5dd758c64f
dcae301662357a4e
a33ca863c357aaa3
69cb9a1d6a3c9503
091771854f3ea9e0
92156730be98a1f0
d6481268e8cfef27
beb3915303cb4270
3065b91097bf0d03
9d0042256683a1d8
5020ddd9719ef01e
3b5e5bbe5afe9cb6
0420b83cfee175a3
98f8b7f109774260
b54b5fdbd45237c2
b976158652d10f9d
79213c91f1cead86
18febd23d8eb18c7
05dd9568dace80cd
d11dde90bb7182f4
8040b45921f1ad40
b9cd8583111c7571
886faf0c211f3500
3d4055655ef960e9
59f184991e582874
3ede8a0698f568f3
4595e91c93f02aaf
ca03c68798ac0dd9
9d919efec88f4d73
87768792c0ddd97d
ba55c74123752951
4e8085bd12d8d27e
a24e3501c693155f
b16baca917bbfc39
0ff86b89da149f3a
6fdbc63733757d44
3e2fa0af023ee0b8
e1cc0aa6ebe921fa
c15ed504079032

In [16]:
def events_per_sub_process(data, days):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-16-2f48aff96ea0>, line 2)